Закоменченные строки пока не удалять (желательно существующие ячейки не изменять, а создавать новые)

In [1]:
# ячейка с импортом необходимого (исключить импорт в других ячейках)
from pyspark.sql import *
from pyspark import SparkContext, SparkConf
import json
import numpy as np
import requests
from pyspark.sql import dataframe
from pyspark.sql.types import *
import pandas as pd
from IPython.display import display, HTML
from pyspark.sql.functions import avg 
from pyspark.mllib.stat import Statistics

In [2]:
# ячека с запуском сессис spark (возможно переделать в функцию)
spark = SparkSession.builder \
    .master("local") \
    .appName("HistData") \
    .config("spark.debug.maxToStringFields", "1000") \
    .getOrCreate()


In [3]:
def getResourceDict (json):
    # по листу метрики из json данных возвращает словарь процессов
    list_data = json['data']['result'] # список словарей по процессам
    # предобработка - convert to float
#     for i in list_data:
#         for j in range(len(i["values"])):
#             i["values"][j][1] = float(i["values"][j][1])
    # values_list = list(map(lambda x: x["values"], list_data))
    resourceTypeDict = dict(map(lambda x: (x["metric"]["resource_type"], x["values"]), list_data))
    # iterObj = iter(resourceTypeDict)
    # print(list(iterObj))
    return resourceTypeDict

In [4]:
# Блок склейки df спарка путем создания новых df в самом спарке (string data)

# импорт временного json по API прометеуса
# response = requests.get('http://localhost:9090/api/v1/query?query=' + 'PIDMemory' + '[24h]')
# listData = response.json()

with open('C:/Users/senio/OneDrive/Рабочий стол/Dunau/MonitoringSystem/query.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

resourceTypeDict = getResourceDict(json_data)
column_list = list(resourceTypeDict.keys())
print("Count (processes x metric); ", len(column_list)) # 
data = list(resourceTypeDict.values())
col = ['Time'] + list(resourceTypeDict.keys())

df = None
for el in column_list[:6]: # изменять срез для получения большего кол-ва столбцов
    # df_schema = StructType([StructField("Time", FloatType(), True), StructField(el, DoubleType(), True)]) - почему-то отторгает такую схему
    if df is None:
        df = spark.createDataFrame(resourceTypeDict[el]).toDF("Time", el)
    else:
        df_right = spark.createDataFrame(resourceTypeDict[el]).toDF("Time", el)
        df = df.join(df_right, "Time", "right")

print(f"#### Columns: {len(df.columns)} #### {df.count()}")
df.printSchema()

Count (processes x metric);  3324
#### Columns: 7 #### 37
root
 |-- Time: double (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_0: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_1: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_10: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_11: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_2: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_3: string (nullable = true)



In [5]:
def printDf(sprkDF): 
    # преобразует спарквский df в пандасовский для нормального визуального отображения
    newdf = sprkDF.toPandas()
    return HTML(newdf.to_html())

Ячейка ниже - копия логики создания объединенного датафрейма но на пандасе (1001 столбец генерит за несколько секунд) c дальнейшим переводом его в df спарка.
Для вывода на экран df спарка использовать функцию printDf 


In [6]:
# response = requests.get('http://localhost:9090/api/v1/query?query=' + 'PIDMemory' + '[24h]')
# listData = response.json()

with open('C:/Users/senio/OneDrive/Рабочий стол/Dunau/MonitoringSystem/query.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

resourceTypeDict = getResourceDict(json_data)
column_list = list(resourceTypeDict.keys())
print(len(column_list))
data = list(resourceTypeDict.values())
col = ['Time'] + list(resourceTypeDict.keys())

pdf = None
for el in column_list[:400]: # изменять срез для получения большего кол-ва столбцов
    if pdf is None:
        pdf = pd.DataFrame(resourceTypeDict[el], columns=["Time", el])
    else:
        pdf_right = pd.DataFrame(resourceTypeDict[el], columns=["Time", el])
        pdf = pdf.join(pdf_right.set_index("Time"), on ="Time")

pdf = pdf.astype(float)
#pdf = pdf.apply(lambda x: x.fillna(x.mean()),axis=0)

spark.conf.set("spark.sql.debug.maxToStringFields", "1000")
df_spark = spark.createDataFrame(pdf)
print(f"#### Columns: {len(pdf.columns)} #### ")
# df_spark.show()


tempList = []   # УБИРАЕТ ТОЧКИ ИЗ НАЗВАНИЙ СТОЛБЦОВ
for col in df_spark.columns:
    new_name = col.strip()
    new_name = "".join(new_name.split())
    new_name = new_name.replace('.','') 
    tempList.append(new_name) 
# print(tempList) #Just for the sake of it 
df_spark = df_spark.toDF(*tempList) 

printDf(df_spark)

3324
#### Columns: 401 #### 


,Time,AGMServiceexe_5208_16045226900_0,AGMServiceexe_5208_16045226900_1,AGMServiceexe_5208_16045226900_10,AGMServiceexe_5208_16045226900_11,AGMServiceexe_5208_16045226900_2,AGMServiceexe_5208_16045226900_3,AGMServiceexe_5208_16045226900_4,AGMServiceexe_5208_16045226900_5,AGMServiceexe_5208_16045226900_6,AGMServiceexe_5208_16045226900_7,AGMServiceexe_5208_16045226900_8,AGMServiceexe_5208_16045226900_9,AGSServiceexe_5228_16045226900_0,AGSServiceexe_5228_16045226900_1,AGSServiceexe_5228_16045226900_10,AGSServiceexe_5228_16045226900_11,AGSServiceexe_5228_16045226900_2,AGSServiceexe_5228_16045226900_3,AGSServiceexe_5228_16045226900_4,AGSServiceexe_5228_16045226900_5,AGSServiceexe_5228_16045226900_6,AGSServiceexe_5228_16045226900_7,AGSServiceexe_5228_16045226900_8,AGSServiceexe_5228_16045226900_9,AcroRd32exe_17332_16047537010_0,AcroRd32exe_17332_16047537010_1,AcroRd32exe_17332_16047537010_10,AcroRd32exe_17332_16047537010_11,AcroRd32exe_17332_16047537010_2,AcroRd32exe_17332_16047537010_3,AcroRd32exe_17332_16047537010_4,AcroRd32exe_17332_16047537010_5,AcroRd32exe_17332_16047537010_6,AcroRd32exe_17332_16047537010_7,AcroRd32exe_17332_16047537010_8,AcroRd32exe_17332_16047537010_9,AcroRd32exe_20736_16047537000_0,AcroRd32exe_20736_16047537000_1,AcroRd32exe_20736_16047537000_10,AcroRd32exe_20736_16047537000_11,AcroRd32exe_20736_16047537000_2,AcroRd32exe_20736_16047537000_3,AcroRd32exe_20736_16047537000_4,AcroRd32exe_20736_16047537000_5,AcroRd32exe_20736_16047537000_6,AcroRd32exe_20736_16047537000_7,AcroRd32exe_20736_16047537000_8,AcroRd32exe_20736_16047537000_9,AdminServiceexe_4788_16045226900_0,AdminServiceexe_4788_16045226900_1,AdminServiceexe_4788_16045226900_10,AdminServiceexe_4788_16045226900_11,AdminServiceexe_4788_16045226900_2,AdminServiceexe_4788_16045226900_3,AdminServiceexe_4788_16045226900_4,AdminServiceexe_4788_16045226900_5,AdminServiceexe_4788_16045226900_6,AdminServiceexe_4788_16045226900_7,AdminServiceexe_4788_16045226900_8,AdminServiceexe_4788_16045226900_9,AdobeNotificationClientexe_16300_16047473370_0,AdobeNotificationClientexe_16300_16047473370_1,AdobeNotificationClientexe_16300_16047473370_10,AdobeNotificationClientexe_16300_16047473370_11,AdobeNotificationClientexe_16300_16047473370_2,AdobeNotificationClientexe_16300_16047473370_3,AdobeNotificationClientexe_16300_16047473370_4,AdobeNotificationClientexe_16300_16047473370_5,AdobeNotificationClientexe_16300_16047473370_6,AdobeNotificationClientexe_16300_16047473370_7,AdobeNotificationClientexe_16300_16047473370_8,AdobeNotificationClientexe_16300_16047473370_9,ApplicationFrameHostexe_9484_16047342190_0,ApplicationFrameHostexe_9484_16047342190_1,ApplicationFrameHostexe_9484_16047342190_10,ApplicationFrameHostexe_9484_16047342190_11,ApplicationFrameHostexe_9484_16047342190_2,ApplicationFrameHostexe_9484_16047342190_3,ApplicationFrameHostexe_9484_16047342190_4,ApplicationFrameHostexe_9484_16047342190_5,ApplicationFrameHostexe_9484_16047342190_6,ApplicationFrameHostexe_9484_16047342190_7,ApplicationFrameHostexe_9484_16047342190_8,ApplicationFrameHostexe_9484_16047342190_9,Codeexe_11148_16047546260_0,Codeexe_11148_16047546260_1,Codeexe_11148_16047546260_10,Codeexe_11148_16047546260_11,Codeexe_11148_16047546260_2,Codeexe_11148_16047546260_3,Codeexe_11148_16047546260_4,Codeexe_11148_16047546260_5,Codeexe_11148_16047546260_6,Codeexe_11148_16047546260_7,Codeexe_11148_16047546260_8,Codeexe_11148_16047546260_9,Codeexe_13984_16047546270_0,Codeexe_13984_16047546270_1,Codeexe_13984_16047546270_10,Codeexe_13984_16047546270_11,Codeexe_13984_16047546270_2,Codeexe_13984_16047546270_3,Codeexe_13984_16047546270_4,Codeexe_13984_16047546270_5,Codeexe_13984_16047546270_6,Codeexe_13984_16047546270_7,Codeexe_13984_16047546270_8,Codeexe_13984_16047546270_9,Codeexe_16796_16047546470_0,Codeexe_16796_16047546470_1,Codeexe_16796_16047546470_10,Codeexe_16796_16047546470_11,Codeexe_16796_16047546470_2,Codeexe_16796_16047546470_3,Codeexe_16796_16047546470_4,Codeexe_16796_16047546470_5,Codeexe_16796_1604

In [32]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#     print(pdf)

In [7]:
def delTime(df):   # удаление столбца Time
    columns_to_drop = ['Time'] 
    df_spark = df.drop(*columns_to_drop)
    return df_spark

In [8]:
def delColumnsWithTheSameValue(df):   # УДАЛЕНИЕ СТОЛБЦОВ С ОДИНАКОВЫМ ЗНАЧЕНИЕМ
    from pyspark.sql.functions import approx_count_distinct               
    count_distinct_df = df.select([approx_count_distinct(x).alias("{0}".format(x)) for x in df.columns])
    dict_of_columns = count_distinct_df.toPandas().to_dict(orient='list')
    # сохранение колонок, в которых только 1 значение
    distinct_columns=[k for k,v in dict_of_columns.items() if v == [1]]

    df_spark = df.drop(*distinct_columns)
    return df_spark

In [9]:
def NanSwupNull(df):  # Замена Nan на Null
    df_spark = df.replace(float('nan'), None) 
    return df_spark

In [10]:
def fill_with_mean(this_df, exclude=set()):   # ЗАМЕНА NULL НА СРЕДНЕЕ ЗНАЧЕНИЕ

    stats = this_df.agg(*(avg(c).alias(c) for c in this_df.columns if c not in exclude))
    return this_df.na.fill(stats.first().asDict())

In [11]:
df_spark = delTime(df_spark)
df_spark = NanSwupNull(df_spark)
df_spark = fill_with_mean(df_spark, [])
df_spark = delColumnsWithTheSameValue(df_spark)

In [12]:
print(f"#### Columns: {len(df_spark.columns)} #### ")
printDf(df_spark) # БЕЗ NULL, БЕЗ КОРРЕЛИРУЮЩИХ СТОЛБЦОВ, БЕЗ ВРЕМЕНИ

#### Columns: 179 #### 


,AGMServiceexe_5208_16045226900_0,AGMServiceexe_5208_16045226900_4,AGSServiceexe_5228_16045226900_0,AGSServiceexe_5228_16045226900_4,AcroRd32exe_17332_16047537010_0,AcroRd32exe_17332_16047537010_2,AcroRd32exe_17332_16047537010_4,AcroRd32exe_20736_16047537000_0,AcroRd32exe_20736_16047537000_2,AcroRd32exe_20736_16047537000_4,AdminServiceexe_4788_16045226900_0,AdminServiceexe_4788_16045226900_2,AdminServiceexe_4788_16045226900_4,AdobeNotificationClientexe_16300_16047473370_0,AdobeNotificationClientexe_16300_16047473370_4,ApplicationFrameHostexe_9484_16047342190_0,ApplicationFrameHostexe_9484_16047342190_2,ApplicationFrameHostexe_9484_16047342190_4,Codeexe_11148_16047546260_0,Codeexe_11148_16047546260_1,Codeexe_11148_16047546260_11,Codeexe_11148_16047546260_2,Codeexe_11148_16047546260_4,Codeexe_11148_16047546260_6,Codeexe_11148_16047546260_8,Codeexe_11148_16047546260_9,Codeexe_13984_16047546270_0,Codeexe_13984_16047546270_2,Codeexe_13984_16047546270_4,Codeexe_16796_16047546470_0,Codeexe_16796_16047546470_1,Codeexe_16796_16047546470_11,Codeexe_16796_16047546470_2,Codeexe_16796_16047546470_4,Codeexe_16796_16047546470_6,Codeexe_16796_16047546470_8,Codeexe_16796_16047546470_9,Codeexe_20032_16047546270_0,Codeexe_20032_16047546270_1,Codeexe_20032_16047546270_10,Codeexe_20032_16047546270_11,Codeexe_20032_16047546270_2,Codeexe_20032_16047546270_4,Codeexe_20032_16047546270_6,Codeexe_20032_16047546270_8,Codeexe_20032_16047546270_9,Codeexe_20260_16047546270_0,Codeexe_20260_16047546270_1,Codeexe_20260_16047546270_11,Codeexe_20260_16047546270_2,Codeexe_20260_16047546270_4,Codeexe_20260_16047546270_6,Codeexe_20260_16047546270_8,Codeexe_20260_16047546270_9,Codeexe_23476_16047546300_0,Codeexe_23476_16047546300_1,Codeexe_23476_16047546300_11,Codeexe_23476_16047546300_2,Codeexe_23476_16047546300_4,Codeexe_23476_16047546300_8,Codeexe_23476_16047546300_9,Codeexe_9648_16047546450_0,Codeexe_9648_16047546450_1,Codeexe_9648_16047546450_11,Codeexe_9648_16047546450_2,Codeexe_9648_16047546450_4,Codeexe_9648_16047546450_5,Codeexe_9648_16047546450_6,Codeexe_9648_16047546450_7,Codeexe_9648_16047546450_8,Codeexe_9648_16047546450_9,CodeHelperexe_21096_16047546460_0,CodeHelperexe_21096_16047546460_1,CodeHelperexe_21096_16047546460_11,CodeHelperexe_21096_16047546460_2,CodeHelperexe_21096_16047546460_4,CodeHelperexe_21096_16047546460_8,CodeHelperexe_21096_16047546460_9,CompPkgSrvexe_20888_16047342860_0,CompPkgSrvexe_20888_16047342860_1,CompPkgSrvexe_20888_16047342860_11,CompPkgSrvexe_20888_16047342860_2,CompPkgSrvexe_20888_16047342860_4,CompPkgSrvexe_20888_16047342860_8,CompPkgSrvexe_20888_16047342860_9,Discordexe_13020_16047624130_0,Discordexe_13020_16047624130_1,Discordexe_13020_16047624130_11,Discordexe_13020_16047624130_2,Discordexe_13020_16047624130_4,Discordexe_13020_16047624130_8,Discordexe_13020_16047624130_9,Discordexe_15308_16047624080_0,Discordexe_15308_16047624080_1,Discordexe_15308_16047624080_11,Discordexe_15308_16047624080_2,Discordexe_15308_16047624080_4,Discordexe_15308_16047624080_6,Discordexe_15308_16047624080_7,Discordexe_15308_16047624080_8,Discordexe_15308_16047624080_9,Discordexe_20344_16047624190_0,Discordexe_20344_16047624190_1,Discordexe_20344_16047624190_11,Discordexe_20344_16047624190_2,Discordexe_20344_16047624190_4,Discordexe_20344_16047624190_6,Discordexe_20344_16047624190_8,Discordexe_20344_16047624190_9,Discordexe_22368_16047624060_0,Discordexe_22368_16047624060_1,Discordexe_22368_16047624060_11,Discordexe_22368_16047624060_2,Discordexe_22368_16047624060_4,Discordexe_22368_16047624060_5,Discordexe_22368_16047624060_6,Discordexe_22368_16047624060_8,Discordexe_22368_16047624060_9,Discordexe_22416_16047624130_0,Discordexe_22416_16047624130_1,Discordexe_22416_16047624130_11,Discordexe_22416_16047624130_2,Discordexe_22416_16047624130_4,Discordexe_22416_16047624130_6,Discordexe_22416_16047624130_7,Discordexe_22416_16047624130_8,Discordexe_22416_16047624130_9,Discordexe_3136_16047624080_0,Discordexe_3136_16047624080_1,Discordexe_3136_16047